In [ ]:
from pycalphad import Model, Database, Workspace, variables as v
from pycalphad.property_framework.metaproperties import IsolatedPhase

import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as spi

from sympy import symbols, preorder_traversal
import sympy as sp
from sympy import Piecewise, Symbol, integrate

db = Database("../../TDDatabaseFiles_temp/alzn_mey.tdb")
comps = ['AL', 'ZN']

In [ ]:
# Pull out the data in numerical form
wks2 = Workspace(db, comps,
                ['FCC_A1', 'HCP_A3', 'LIQUID'],
                {v.X('ZN'):(0,1,0.02), v.T: 600, v.P:101325, v.N: 1})

fig = plt.figure()
ax = fig.add_subplot()

x = wks2.get(v.X('ZN'))
ax.set_xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")

for phase_name in wks2.phases:
    metastable_wks = wks2.copy()
    metastable_wks.phases = [phase_name]
    prop = IsolatedPhase(phase_name, metastable_wks)(f'GM({phase_name})')
    prop.display_name = f'GM({phase_name})'
    ax.plot(x, wks2.get(prop), label=prop.display_name)
ax.legend()

plt.show()

In [ ]:
# We are going to get the energy function for a single phase and compare to the numerical data
phase_name = 'HCP_A3'
model = Model(db, comps, phase_name)

expr = sp.sympify(model.GM)
# display(expr)

# Extract symbols while preserving order of first occurrence
ordered_symbols = []
seen = set()

for node in preorder_traversal(expr):
    if node.is_Symbol and node not in seen:
        ordered_symbols.append(node)
        seen.add(node)

# print(ordered_symbols)

# Evaluate the function at some temperature
temperature = 600
energy_spline = expr.subs(v.T, temperature)

# Replace the Al fraction with 1-Zn
energy_spline = energy_spline.subs(ordered_symbols[0], 1-ordered_symbols[1]).evalf()

display(energy_spline)
# plot from 0 to 1 on the composition axis
y_range = np.arange(0, 1, 0.01)
energy_values = [float(energy_spline.subs(ordered_symbols[1], y_val)) for y_val in y_range]

plt.plot(y_range, energy_values, label='GM(HCP_A3) spline')
plt.xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")
plt.legend()
plt.show()

In [ ]:
x_data = y_range
y_true = np.array(energy_values)

A = np.column_stack([x_data**k for k in range(8)])  # columns: x^0, x^1, ..., x^7
y_data = y_true - (x_data - 1.0)**8

c_lstsq, residuals, rank, s = np.linalg.lstsq(A, y_data, rcond=None)

print("Coefficients (c_0 ... c_7):", c_lstsq)
print("Residual sum of squares:", residuals)

# ------------------------------------------------
# Evaluate polynomial and plot
# ------------------------------------------------
x_plot = np.linspace(-1, 2, 200)

poly_terms = sum(c_lstsq[k] * x_plot**k for k in range(8))
high_order_term = (x_plot - 1.0)**8

y_plot = poly_terms + high_order_term

# Plot the result
plt.figure(figsize=(8, 5))
plt.plot(x_plot, y_plot, label="Fitted Polynomial", color="b")
plt.scatter(x_data, y_true, color="r", label="True Data", zorder=1, s = 10)
plt.axvline(1, linestyle="--", color="gray", label="x = 1")
plt.ylim(2*max(y_true), .5*min(y_true))
plt.xlabel("x")
plt.ylabel("P(x)")
plt.legend()
plt.title("Polynomial Fit with Diverging End Behavior at x=1")
plt.grid(True)
plt.show()

In [ ]:
# Here we are going to projedt a piecewise function onto a polynomial basis using Galerkin's method

x = Symbol('x', real=True)
f_piecewise = Piecewise(
    (x + 1, (x >= 1)), 
    (4*x**2, (x < 1))
)

degree = 8
basis = [x**i for i in range(degree)]

M = sp.zeros(degree, degree)
for i in range(degree):
    for j in range(degree):
        M[i,j] = integrate(basis[i]*basis[j], (x, 0, 2))


b_vec = sp.zeros(degree,1)
for i in range(degree):
    b_vec[i,0] = integrate(basis[i]*f_piecewise, (x, 0, 2))

c = M.LUsolve(b_vec)

P = sp.simplify(sum(c[i,0]*basis[i] for i in range(degree)))
display(P)

In [ ]:
f_piecewise_lambda = sp.lambdify(x, f_piecewise, 'numpy')
P_fitted_lambda = sp.lambdify(x, P, 'numpy')

x_vals = np.linspace(-0.2, 2.2, 400)
y_piecewise = f_piecewise_lambda(x_vals)
y_fitted = P_fitted_lambda(x_vals)

plt.figure(figsize=(8, 5))
plt.plot(x_vals, y_piecewise, label="Original Piecewise Function", linestyle="dashed", color="r")
plt.plot(x_vals, y_fitted, label="Least Squares Fitted Polynomial", color="b")
plt.axvline(0, linestyle="--", color="gray", lw=1)
plt.axvline(1, linestyle="--", color="gray", lw=1)
plt.axvline(2, linestyle="--", color="gray", lw=1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title("Least Squares Polynomial Approximation of a Piecewise Function")
plt.legend()
plt.ylim(-2, 5)
plt.grid(True)
plt.show()

In [ ]:
# We are going to get the energy function for a single phase and compare to the numerical data
phase_name = 'HCP_A3'
model = Model(db, comps, phase_name)

expr = sp.sympify(model.GM)
# display(expr)

# Extract symbols while preserving order of first occurrence
ordered_symbols = []
seen = set()

for node in preorder_traversal(expr):
    if node.is_Symbol and node not in seen:
        ordered_symbols.append(node)
        seen.add(node)

# print(ordered_symbols)

# Evaluate the function at some temperature
temperature = 600
energy_spline = expr.subs(v.T, temperature)

# Replace the Al fraction with 1-Zn
energy_spline = energy_spline.subs(ordered_symbols[0], 1-ordered_symbols[1]).evalf()

# Sub out the composition variable
energy_spline = energy_spline.subs(ordered_symbols[1], x)
display(energy_spline)

In [ ]:
degree = 8
basis = [x**i for i in range(degree)]

M = sp.zeros(degree, degree)
for i in range(degree):
    for j in range(degree):
        M[i,j] = integrate(basis[i]*basis[j], (x, 0, 1))
        
b_vec = sp.zeros(degree,1)
for i in range(degree):
    # We need to make the product function numerical to integrate
    lambda_func = sp.lambdify(x, basis[i]*energy_spline, 'numpy')
    b_vec[i,0] = spi.quad(lambda_func, 0, 1)[0] # Take only the value and drop the error term 

c = M.LUsolve(b_vec)

P = sp.simplify(sum(c[i,0]*basis[i] for i in range(degree)))
display(P)

In [ ]:

f_piecewise_lambda = sp.lambdify(x, energy_spline, 'numpy')
P_fitted_lambda = sp.lambdify(x, P, 'numpy')

# Extend the range slightly beyond [0,1]
x_vals = np.linspace(-1, 2, 400)

y_piecewise = f_piecewise_lambda(x_vals)
y_fitted = P_fitted_lambda(x_vals)

plt.figure(figsize=(8, 5))
plt.plot(x_vals, y_piecewise, label="Original Piecewise Function", linestyle="dashed", color="r")
plt.plot(x_vals, y_fitted, label="Least Squares Fitted Polynomial", color="b")
plt.axvline(0, linestyle="--", color="gray", lw=1)
plt.axvline(1, linestyle="--", color="gray", lw=1)
plt.axvline(2, linestyle="--", color="gray", lw=1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title("Least Squares Polynomial Approximation of a Piecewise Function")

plt.xlim(-.2, 1.2)
plt.ylim(-30000, -13000)

plt.legend()
plt.xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")
plt.grid(True)
plt.show()

In [ ]:
def fit_polynomial_with_Galerkin(f_peicewise, highest_order, domain):
    x = f_peicewise.free_symbols.pop()
    basis = [x**i for i in range(highest_order)]

    M = sp.zeros(highest_order, highest_order)
    for i in range(highest_order):
        for j in range(highest_order):
            lambda_func = sp.lambdify(x, basis[i]*basis[j], 'numpy')
            M[i,j] =  spi.quad(lambda_func, (x, domain[0], domain[1]))
            
    b_vec = sp.zeros(highest_order,1)
    for i in range(highest_order):
        # We need to make the product function numerical to integrate
        lambda_func = sp.lambdify(x, basis[i]*energy_spline, 'numpy')
        b_vec[i,0] = spi.quad(lambda_func, domain[0], domain[1])[0] # Take only the value and drop the error term 

    c = M.LUsolve(b_vec)

    P = sp.simplify(sum(c[i,0]*basis[i] for i in range(highest_order)))
    display(P)

In [ ]:
fit_polynomial_with_Galerkin(energy_spline, 8, (0, 1))

In [ ]:
energy_spline.free_symbols.pop()